In [15]:
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

class SkywayTester:
    def __init__(self, base_url):
        self.base_url = base_url
        self.setup_driver()
        
    def setup_driver(self):
        """Setup the Chrome WebDriver with appropriate options"""
        chrome_options = Options()
        # Uncomment the following line to run in headless mode
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        
        # Setup WebDriver
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
    
    def test_registration(self, first_name, last_name, username, email, password):
        """Test the registration functionality"""
        try:
            # Navigate to the registration page - use direct path
            print(f"[{self.get_formatted_time()}] Navigating to registration page...")
            self.driver.get(f"{self.base_url}register.html")  # No slash before register.html
            
            # Wait for the page to load completely
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "registerForm"))
            )
            
            # Log the current page title
            print(f"[{self.get_formatted_time()}] Page title: {self.driver.title}")
            
            # Fill in the registration form
            print(f"[{self.get_formatted_time()}] Entering registration details...")
            self.driver.find_element(By.ID, "firstName").send_keys(first_name)
            self.driver.find_element(By.ID, "lastName").send_keys(last_name)
            self.driver.find_element(By.ID, "registerUsername").send_keys(username)
            self.driver.find_element(By.ID, "email").send_keys(email)
            self.driver.find_element(By.ID, "registerPassword").send_keys(password)
            self.driver.find_element(By.ID, "confirmPassword").send_keys(password)
            
            # Check the terms agreement checkbox
            terms_checkbox = self.driver.find_element(By.ID, "termsAgree")
            if not terms_checkbox.is_selected():
                terms_checkbox.click()
                print(f"[{self.get_formatted_time()}] Checked 'Terms and Conditions'")
            
            # Take screenshot before submission
            self.driver.save_screenshot("skyway_before_registration.png")
            print(f"[{self.get_formatted_time()}] Screenshot saved before registration")
            
            # Submit the form
            print(f"[{self.get_formatted_time()}] Submitting registration form...")
            submit_button = self.driver.find_element(By.XPATH, "//button[contains(text(), 'Create Account')]")
            submit_button.click()
            
            # Wait for registration confirmation
            try:
                # Check for success alert
                alert_element = WebDriverWait(self.driver, 10).until(
                    EC.visibility_of_element_located((By.ID, "registerAlert"))
                )
                
                if "successful" in alert_element.text.lower():
                    print(f"[{self.get_formatted_time()}] Registration successful! Message: {alert_element.text}")
                    
                    # Wait for redirection to login page
                    WebDriverWait(self.driver, 10).until(
                        EC.url_contains("login.html")
                    )
                    print(f"[{self.get_formatted_time()}] Redirected to login page: {self.driver.current_url}")
                    
                    return True
                else:
                    print(f"[{self.get_formatted_time()}] Registration may have failed. Alert message: {alert_element.text}")
                    return False
            except:
                print(f"[{self.get_formatted_time()}] Registration status unknown. Current URL: {self.driver.current_url}")
                return False
                
        except Exception as e:
            print(f"[{self.get_formatted_time()}] Registration test failed with error: {str(e)}")
            return False
    
    def test_login(self, username, password):
        """Test the login functionality"""
        try:
            # Navigate to the login page - use direct path
            print(f"[{self.get_formatted_time()}] Navigating to login page...")
            self.driver.get(f"{self.base_url}login.html")  # No slash before login.html
            
            # Wait for the page to load completely
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "loginForm"))
            )
            
            # Log the current page title
            print(f"[{self.get_formatted_time()}] Page title: {self.driver.title}")
            
            # Fill in the username field
            print(f"[{self.get_formatted_time()}] Entering username: {username}")
            username_input = self.driver.find_element(By.ID, "username")
            username_input.clear()
            username_input.send_keys(username)
            
            # Fill in the password field
            print(f"[{self.get_formatted_time()}] Entering password")
            password_input = self.driver.find_element(By.ID, "password")
            password_input.clear()
            password_input.send_keys(password)
            
            # Optional: Check "Remember me" checkbox
            remember_checkbox = self.driver.find_element(By.ID, "rememberMe")
            if not remember_checkbox.is_selected():
                remember_checkbox.click()
                print(f"[{self.get_formatted_time()}] Checked 'Remember me'")
            
            # Take screenshot before submission
            self.driver.save_screenshot("skyway_before_login.png")
            print(f"[{self.get_formatted_time()}] Screenshot saved before login")
            
            # Submit the form
            print(f"[{self.get_formatted_time()}] Submitting login form...")
            submit_button = self.driver.find_element(By.XPATH, "//button[contains(text(), 'Login')]")
            submit_button.click()
            
            # Wait for login to complete - look for welcome message or redirect
            try:
                # Check for success alert first
                alert_element = WebDriverWait(self.driver, 5).until(
                    EC.visibility_of_element_located((By.ID, "loginAlert"))
                )
                
                if "successful" in alert_element.text.lower():
                    print(f"[{self.get_formatted_time()}] Login alert shown: {alert_element.text}")
                    
                    # Wait for redirection to dashboard
                    WebDriverWait(self.driver, 10).until(
                        EC.url_contains("dashboard.html")
                    )
                    print(f"[{self.get_formatted_time()}] Login successful! Redirected to: {self.driver.current_url}")
                    return True
            except:
                # If no alert, check if we've been redirected to dashboard or welcome message appears
                try:
                    WebDriverWait(self.driver, 10).until(
                        lambda driver: "dashboard.html" in driver.current_url or 
                        driver.find_element(By.ID, "welcomeUser").is_displayed()
                    )
                    print(f"[{self.get_formatted_time()}] Login successful! Current URL: {self.driver.current_url}")
                    return True
                except:
                    # Check if there's an error message
                    try:
                        error_element = self.driver.find_element(By.ID, "loginAlert")
                        print(f"[{self.get_formatted_time()}] Login failed. Error: {error_element.text}")
                    except:
                        print(f"[{self.get_formatted_time()}] Login status unknown. Current URL: {self.driver.current_url}")
                    return False
                
        except Exception as e:
            print(f"[{self.get_formatted_time()}] Login test failed with error: {str(e)}")
            return False
            

  
    def get_formatted_time(self):
        """Get current time in specified format"""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
    def close(self):
        """Close the WebDriver"""
        if self.driver:
            self.driver.quit()
            print(f"[{self.get_formatted_time()}] WebDriver closed")


if __name__ == "__main__":
    # Configuration
    # For local files - Make sure to include trailing slash if needed for your file structure
    BASE_URL = "C:\\Users\\USER\\Desktop\\flight-booking\\"  # Change this to the path where your HTML files are located
    
    # Test credentials
    # Use your predefined user or create a new one
    TEST_FIRST_NAME = "Nzho"
    TEST_LAST_NAME = "Tester"
    TEST_USERNAME = "Nzho7test"  # Modified to avoid conflicts with your existing user
    TEST_EMAIL = "nzho7test@example.com"
    TEST_PASSWORD = "Password123"
    
    # Create and run the tester
    tester = SkywayTester(BASE_URL)
    
    # Set current time for testing
    current_time = "2025-04-21 17:13:59"
    print(f"Current Date and Time for Testing: {current_time}")
    
    try:
        # Test registration
        print("\n" + "="*50)
        print(f"[{tester.get_formatted_time()}] STARTING REGISTRATION TEST")
        print("="*50)
        reg_success = tester.test_registration(TEST_FIRST_NAME, TEST_LAST_NAME, 
                                            TEST_USERNAME, TEST_EMAIL, TEST_PASSWORD)
        
        if reg_success:
            time.sleep(2)  # Short pause to see the registration success state
        
        # Test login
        print("\n" + "="*50)
        print(f"[{tester.get_formatted_time()}] STARTING LOGIN TEST")
        print("="*50)
        login_success = tester.test_login(TEST_USERNAME, TEST_PASSWORD)
        
        
    finally:
        # Always close the driver to prevent resource leaks
        time.sleep(3)  # Keep browser open for a moment to see results
        tester.close()

Current Date and Time for Testing: 2025-04-21 17:13:59

[2025-04-21 21:00:31] STARTING REGISTRATION TEST
[2025-04-21 21:00:31] Navigating to registration page...
[2025-04-21 21:00:32] Page title: Sign Up - SkyWay
[2025-04-21 21:00:32] Entering registration details...
[2025-04-21 21:00:32] Checked 'Terms and Conditions'
[2025-04-21 21:00:33] Screenshot saved before registration
[2025-04-21 21:00:33] Submitting registration form...
[2025-04-21 21:00:33] Registration successful! Message: Registration successful! Redirecting to login...
[2025-04-21 21:00:34] Redirected to login page: file:///C:/Users/USER/Desktop/flight-booking/login.html

[2025-04-21 21:00:37] STARTING LOGIN TEST
[2025-04-21 21:00:37] Navigating to login page...
[2025-04-21 21:00:37] Page title: Login - SkyWay
[2025-04-21 21:00:37] Entering username: Nzho7test
[2025-04-21 21:00:37] Entering password
[2025-04-21 21:00:37] Checked 'Remember me'
[2025-04-21 21:00:37] Screenshot saved before login
[2025-04-21 21:00:37] Submit